## Problem 3. COVID-19 database  exploration (30 points)

In this problem, you are expected to explore the DRKG (Drug Repurposing Knowledge Graph) databse, which includes the genes, diseases and drugs for COVID-19. Firstly, you need to build a graph attention neural network (GAT) to predict the weights of small molecules as a pre-trained model. Secondly, you should build the RotatE model to recommend drugs by the pre-trained embedding vectors.

### Requirements:

3.1 Preprocess the dataset for the weight prediction and explain your reasons (5 points).

3.2 Build the GAT model to predict the weights of the small molecules. You should build a hidden layer between the GAT module and the final output layer. This hidden layer should extract and output the drug embedding vectors of the drugs. If you don't use a GAT module, you will get 0 point (15 points).

3.3 Collect data from different sources, including the entity embeddings, gene embeddings, and relation embeddings (5 points).

3.4 Output two gene-drug pairs with the lowest and highest scores respectively, by building the RotatE model to check clinical trial drugs for each gene (5 points).

Note：You should write comments to explain what you have done for the important parts and why your code works for the above requirements. All the code and results should be shown in problem3.ipynb (this file).

### References:

The DRKG database: https://github.com/gnn4dr/DRKG.

An example Jupyter notebook is: https://github.com/gnn4dr/DRKG/blob/master/drug_repurpose/COVID-19_drug_repurposing_via_genes.ipynb

3.1 Preprocess the dataset for the weight prediction and explain your reasons

In [1]:
# 读文件
import numpy as np
f1 = open('drugbank_smiles.txt')
drugbank_smiles=f1.read()
f2 = open('drugbank_weight.txt')
drugbank_weight=f2.read()

In [2]:
# 用换行符切分开然后把最后一个空的删掉
drugbank_smiles_list = drugbank_smiles.split('\n')
drugbank_smiles_list2=[i.split('\t') for i in drugbank_smiles_list][0:-1]
print(len(drugbank_smiles_list2))

8807


In [3]:
# 先用换行符切分开，然后把第一行的label删掉
drugbank_weight_list = drugbank_weight.split('\n')
drugbank_weight_list2=[i.split('\t') for i in drugbank_weight_list]
drugbank_weight_list3=drugbank_weight_list2[1:-1]
print(len(drugbank_weight_list3))

9705


In [4]:
# 比对weight和没smiles的，只取有smiles的那些weight，观察weight数据本身也能看出值为-1的那些就是没smiles的，但是还是自己处理一下
# 语法糖，写时一时爽，读时火葬场
# 还是注释一下，i和j都是长度为2的list
drugbank_smiles_list2_0=[j[0] for j in drugbank_smiles_list2]
drugbank_smiles_list2_1=[j[1] for j in drugbank_smiles_list2]
drugbank_smiles_list_end=drugbank_smiles_list2_1
drugbank_weight_list_end=[i[1] for i in drugbank_weight_list3 if i[0] in drugbank_smiles_list2_0]
print(len(drugbank_smiles_list_end))
print(len(drugbank_weight_list_end))

8807
8807


SOL3.1 这么处理数据的原因就是为了找到那些有smiles的小分子，并且找到他们对应的weight这样才能使用数据进行学习和预测，随后对关系图进行相同的匹配，删去无效项。

3.2 Build the GAT model to predict the weights of the small molecules. You should build a hidden layer between the GAT module and the final output layer. This hidden layer should extract and output the drug embedding vectors of the drugs. If you don't use a GAT module, you will get 0 point (15 points).